In [4]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')
import seaborn as sns
import numpy as np
from scipy.stats import norm, skew
from sklearn.metrics import mean_squared_error

In [70]:
x_train = pd.read_csv('train_mod.csv')
x_test = pd.read_csv('test_mod.csv')
y_train = pd.read_csv('y_train.csv')
df_test = pd.read_csv('test.csv')

In [6]:
from sklearn.model_selection import GridSearchCV

# Linear models

In [116]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor 
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingRegressor
import xgboost as xgb
from sklearn.svm import SVR

In [24]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
ridge = Ridge()
lasso = Lasso()
ElNet = ElasticNet()
gbm = GradientBoostingRegressor()

In [28]:
# Ridge
para_ridge = {"alpha": np.logspace(0.8,1.1,100)}
gs_ridge = GridSearchCV(ridge, para_ridge, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
gs_ridge.fit(x_train, y_train)
gs_ridge.best_params_
#{'alpha': 9.930467558623954}

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.32007e-249): result may not be accurate.
  overwrite_a=True).T


{'alpha': 9.930467558623954}

In [157]:
ridge = make_pipeline(RobustScaler(), Ridge(alpha =gs_ridge.best_params_['alpha'], random_state=1))
ridge.fit(x_train, y_train)
score_ridge = ridge.score(x_train, y_train)

In [158]:
rmse_ridge = np.sqrt(mean_squared_error(y_train,ridge.predict(x_train)))
ridge_pred = np.expm1(ridge.predict(x_test))

In [19]:
# Lasso
para_lasso = {"alpha": np.logspace(-4,-3.3,100)}
lasso.set_params(max_iter = 10000)
gs_lasso = GridSearchCV(lasso, para_lasso, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
gs_lasso.fit(x_train, y_train)
gs_lasso.best_params_
#{'alpha': 0.0003503034741265338}

{'alpha': 0.0003503034741265338}

In [104]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =gs_lasso.best_params_['alpha'], random_state=1))
lasso.fit(x_train, y_train)
score_lasso = lasso.score(x_train, y_train)

In [106]:
rmse_lasso = np.sqrt(mean_squared_error(y_train,lasso.predict(x_train)))
lasso_pred = np.expm1(lasso.predict(x_test))

0.0986325629289788

In [47]:
# ElasticNet
para_ElNet = {"alpha": np.logspace(-3.8,-3.3,10),"l1_ratio": np.linspace(0.7,0.9,10)}
ElNet.set_params(max_iter = 5000)
gs_ElNet = GridSearchCV(ElNet, para_ElNet, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
gs_ElNet.fit(x_train, y_train)
gs_ElNet.best_params_
#{'alpha': 0.0003880510732210184, 'l1_ratio': 0.9}

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.110620226919156, tolerance: 0.023278539201380233
  positive)


{'alpha': 0.0003880510732210184, 'l1_ratio': 0.9}

In [122]:
ElNet.set_params(alpha = gs_ElNet.best_params_['alpha'],l1_ratio=gs_ElNet.best_params_['l1_ratio'])
ElNet.fit(x_train, y_train)
score_ElNet = ElNet.score(x_train, y_train)

In [123]:
rmse_ElNet = np.sqrt(mean_squared_error(y_train,ElNet.predict(x_train)))
#(np.sum((y_train.values - ElNet.predict(x_train).reshape(-1,1))**2)/len(y_train))**0.5

In [124]:
obj = pd.DataFrame([[score_ridge,rmse_ridge],[score_lasso,rmse_lasso],[score_ElNet,rmse_ElNet]],\
                   columns = ['score','RMSE'],index=['Ridge','Lasso','ElNet'])

In [125]:
obj

,score,RMSE
Ridge,0.941291,0.096817
Lasso,0.939068,0.098633
ElNet,0.939095,0.098611


In [20]:
obj

,score,RMSE
Ridge,0.946037,0.092320
Lasso,0.943563,0.094412
ElNet,0.946118,0.092251


In [107]:
sub = pd.DataFrame()
sub['Id'] = df_test.Id
sub['SalePrice'] = lasso_pred
sub.to_csv('submission.csv',index=False)

In [75]:
np.arange(1000,10000,1000)

array([1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000])

In [83]:
para_gbm = {"n_estimators":np.arange(1000,10000,1000),'learning_rate':[0.05,0.1,0.5]}
gs_gbm = GridSearchCV(gbm, para_gbm, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
gs_gbm.fit(x_train, y_train)
gs_gbm.best_params_

Exception in thread QueueManagerThread:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py", line 747, in _queue_management_worker
    recursive_terminate(p)
  File "/usr/local/lib/python3.7/site-packages/joblib/externals/loky/backend/utils.py", line 28, in recursive_terminate
    _recursive_terminate_without_psutil(process)
  File "/usr/local/lib/python3.7/site-packages/joblib/externals/loky/backend/utils.py", line 53, in _recursive_terminate_without_psutil
    _recursive_terminate(process.pid)
  File "/usr/local/lib/python3.7/site-packages/joblib/externals/loky/backend/utils.py", line 94, in 

KeyboardInterrupt: 

In [82]:
gbm = GradientBoostingRegressor(max_depth=4, max_features='log2',
                                min_samples_leaf=15, min_samples_split=10, 
                                loss='huber')

In [100]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(x_train, y_train)
score_GBoost = GBoost.score(x_train.values, y_train)
rmse_GBoost = np.sqrt(mean_squared_error(y_train,GBoost.predict(x_train.values)))
GBoost_pred = np.expm1(GBoost.predict(x_test))
rmse_GBoost

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.049770047448252516

In [98]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(x_train.values)
    rmse= np.sqrt(-cross_val_score(model, x_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [99]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Gradient Boosting score: 0.1160 (0.0086)



In [133]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_xgb.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4603, gamma=0.0468,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1.7817, missing=None,
             n_estimators=2200, n_jobs=1, nthread=-1, objective='reg:linear',
             random_state=7, reg_alpha=0.464, reg_lambda=0.8571,
             scale_pos_weight=1, seed=None, silent=1, subsample=0.5213,
             verbosity=1)

In [108]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Xgboost score: 0.1155 (0.0072)



In [170]:
svr = SVR(epsilon = 0.1, gamma=1e-7,C=100000)
svr.fit(x_train,y_train) 
score_svr = svr.score(x_train, y_train)
rmse_svr = np.sqrt(mean_squared_error(y_train,svr.predict(x_train.values)))
svr_pred = np.expm1(svr.predict(x_test))
rmse_svr

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.09550542786876194

In [171]:
stackmodel = StackingRegressor(
        regressors=[ElNet, lasso, ridge, svr, model_xgb, GBoost],
        meta_regressor=SVR(kernel='rbf')
    )

In [ ]:
stackmodel.fit(x_train, y_train)

stacked = stackmodel.predict(x_test)

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [167]:
ensembled = np.expm1((0.2 * ElNet.predict(x_test)) +
                     (0.2 * lasso.predict(x_test)) +
                     (0.2 * model_xgb.predict(x_test)) +
                     (0.2 * GBoost.predict(x_test)) +
                     (0.2 * stacked))

In [168]:
ensembled.shape

(1459,)

In [153]:
stacked.shape

(1459,)

In [154]:
ridge.predict(x_test).shape

(1459, 1)

In [169]:
sub = pd.DataFrame()
sub['Id'] = df_test.Id
sub['SalePrice'] = ensembled
sub.to_csv('submission.csv',index=False)